## Train a character-level GPT on some text data

The inputs here are simple text files, which we chop up to individual characters and then train GPT on. So you could say this is a char-transformer instead of a char-rnn. Doesn't quite roll off the tongue as well. In this example we will feed it some Shakespeare, which we'll get it to predict character-level.

In [1]:
from allennlp.data.token_indexers import TokenIndexer, PretrainedTransformerIndexer
from allennlp.data.tokenizers import Token, Tokenizer, PretrainedTransformerTokenizer

import nltk
#nltk.download('punkt')
import numpy as np
from os import listdir
from os.path import join as pathjoin
import torch
import torch.nn as nn
from torch.nn import functional as F
import tqdm

from minGPT.mingpt.model import GPT, GPTConfig
from minGPT.mingpt.trainer import Trainer, TrainerConfig
# make deterministic
from minGPT.mingpt.utils import sample, set_seed
set_seed(42)

In [2]:
DATA_DIR = '/home/mlepekhin/data/big'
#MODELS_DIR = '/home/mlepekhin/models/big'
transformer_model = 'bert-base-cased'

In [3]:
import math
from torch.utils.data import Dataset


def detokenize(tokens):
    return ' '.join([str(x) for x in tokens[1:-1]]).replace(' ##', '')

class BPEDataset(Dataset):
    def __init__(self, data, block_size):
        chars = sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)
        print('data has %d characters, %d unique.' % (data_size, vocab_size))
        
        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data
    
    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        # grab a chunk of (block_size + 1) characters from the data
        chunk = self.data[idx:idx + self.block_size + 1]
        # encode every character to an integer
        dix = [self.stoi[s] for s in chunk]
        """
        arrange data and targets so that the first i elements of x
        will be asked to predict the i-th element of y. Notice that
        the eventual language model will actually make block_size
        individual predictions at the same time based on this data,
        so we are being clever and amortizing the cost of the forward
        pass of the network. So for example if block_size is 4, then
        we could e.g. sample a chunk of text "hello", the integers in
        x will correspond to "hell" and in y will be "ello". This will
        then actually "multitask" 4 separate examples at the same time
        in the language model:
        - given just "h", please predict "e" as next
        - given "he" please predict "l" next
        - given "hel" predict "l" next
        - given "hell" predict "o" next
        
        In addition, because the DataLoader will create batches of examples,
        every forward/backward pass during traning will simultaneously train
        a LOT of predictions, amortizing a lot of computation. In particular,
        for a batched input of integers X (B, T) where B is batch size and
        T is block_size and Y (B, T), the network will during training be
        simultaneously training to make B*T predictions, all at once! Of course,
        at test time we can paralellize across batch B, but unlike during training
        we cannot parallelize across the time dimension T - we have to run
        a forward pass of the network to recover the next single character of the 
        sequence along each batch dimension, and repeatedly always feed in a next
        character to get the next one.
        
        So yes there is a big asymmetry between train/test time of autoregressive
        models. During training we can go B*T at a time with every forward pass,
        but during test time we can only go B at a time, T times, with T forward 
        passes.
        """
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long)
        return x, y


In [4]:
block_size = 128
tokenizer = PretrainedTransformerTokenizer(transformer_model)
#indexer = PretrainedTransformerIndexer(transformer_model)

In [5]:
def train_gpt_generator(train_text_file, state_dict_file, n_layer=8, n_head=8, n_embd=512,
                        max_epochs=1, batch_size=256):
    text_sentences = nltk.tokenize.sent_tokenize(open(train_text_file, 'r').read())
    tokens = np.concatenate([tokenizer.tokenize(sent)[1:-1] for sent in text_sentences])
    tokens = [str(token) for token in tokens]
    train_dataset = BPEDataset(tokens, block_size) 
    
    mconf = GPTConfig(
        train_dataset.vocab_size, train_dataset.block_size,
        n_layer=n_layer, n_head=n_head, n_embd=n_embd
    )
    model = GPT(mconf)
    tconf = TrainerConfig(
        max_epochs=max_epochs, batch_size=batch_size, learning_rate=6e-4,
        lr_decay=True, warmup_tokens=batch_size*20, final_tokens=2*len(train_dataset)*block_size,
        num_workers=4
    )
    trainer = Trainer(model, train_dataset, None, tconf)
    trainer.train()
    torch.save(model.state_dict(), state_dict_file)

In [6]:
GENRE_DATA_DIR = '/home/mlepekhin/data/big/genre'
GPT_MODELS_DIR = '/home/mlepekhin/models/mini_gpt_big_bpe/'
LANG = 'en'

In [7]:
#train_gpt_generator(
#        pathjoin(GENRE_DATA_DIR, LANG, 'A1.txt'),
#        pathjoin(GPT_MODELS_DIR, LANG, 'A1')
#)

In [ ]:
for train_text_file in tqdm.tqdm(listdir(pathjoin(GENRE_DATA_DIR, LANG))):
    label = train_text_file[:-4]
    train_gpt_generator(
        pathjoin(GENRE_DATA_DIR, LANG, train_text_file),
        pathjoin(GPT_MODELS_DIR, LANG, label)
    )

  0%|          | 0/11 [00:00<?, ?it/s]

data has 1353356 characters, 22161 unique.



  0%|          | 0/5287 [00:00<?, ?it/s]/home/mlepekhin/anaconda3/envs/mlepekhin_research/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '

epoch 1 iter 0: train loss 10.11153. lr 6.000000e-04:   0%|          | 0/5287 [00:07<?, ?it/s]
epoch 1 iter 0: train loss 10.11153. lr 6.000000e-04:   0%|          | 1/5287 [00:07<10:59:54,  7.49s/it]
epoch 1 iter 1: train loss 9.40282. lr 6.000000e-04:   0%|          | 1/5287 [00:07<10:59:54,  7.49s/it] 
epoch 1 iter 1: train loss 9.40282. lr 6.000000e-04:   0%|          | 2/5287 [00:07<7:54:05,  5.38s/it] 
epoch 1 iter 2: train loss 8.94643. lr 5.999999e-04:   0%|          | 2/5287 [00:08<7:54:05,  5.38s/it]
epoch 1 iter 2: train loss 8.94643. lr 5.999999e-04:   0%|          | 3/5287 [00:08<5:43:39,  3.90s/it]
epoch 1 iter 3: train loss

epoch 1 iter 36: train loss 6.62182. lr 5.999820e-04:   1%|          | 37/5287 [00:23<39:36,  2.21it/s]
epoch 1 iter 37: train loss 6.65813. lr 5.999810e-04:   1%|          | 37/5287 [00:24<39:36,  2.21it/s]
epoch 1 iter 37: train loss 6.65813. lr 5.999810e-04:   1%|          | 38/5287 [00:24<39:36,  2.21it/s]
epoch 1 iter 38: train loss 6.71436. lr 5.999800e-04:   1%|          | 38/5287 [00:24<39:36,  2.21it/s]
epoch 1 iter 38: train loss 6.71436. lr 5.999800e-04:   1%|          | 39/5287 [00:24<39:38,  2.21it/s]
epoch 1 iter 39: train loss 6.61231. lr 5.999790e-04:   1%|          | 39/5287 [00:25<39:38,  2.21it/s]
epoch 1 iter 39: train loss 6.61231. lr 5.999790e-04:   1%|          | 40/5287 [00:25<39:36,  2.21it/s]
epoch 1 iter 40: train loss 6.63065. lr 5.999779e-04:   1%|          | 40/5287 [00:25<39:36,  2.21it/s]
epoch 1 iter 40: train loss 6.63065. lr 5.999779e-04:   1%|          | 41/5287 [00:25<39:37,  2.21it/s]
epoch 1 iter 41: train loss 6.59261. lr 5.999768e-04:   1%|     

epoch 1 iter 74: train loss 6.22153. lr 5.999258e-04:   1%|▏         | 75/5287 [00:41<39:48,  2.18it/s]
epoch 1 iter 75: train loss 6.25726. lr 5.999238e-04:   1%|▏         | 75/5287 [00:41<39:48,  2.18it/s]
epoch 1 iter 75: train loss 6.25726. lr 5.999238e-04:   1%|▏         | 76/5287 [00:41<39:38,  2.19it/s]
epoch 1 iter 76: train loss 6.22580. lr 5.999218e-04:   1%|▏         | 76/5287 [00:42<39:38,  2.19it/s]
epoch 1 iter 76: train loss 6.22580. lr 5.999218e-04:   1%|▏         | 77/5287 [00:42<39:33,  2.19it/s]
epoch 1 iter 77: train loss 6.19895. lr 5.999197e-04:   1%|▏         | 77/5287 [00:42<39:33,  2.19it/s]
epoch 1 iter 77: train loss 6.19895. lr 5.999197e-04:   1%|▏         | 78/5287 [00:42<39:28,  2.20it/s]
epoch 1 iter 78: train loss 6.17545. lr 5.999177e-04:   1%|▏         | 78/5287 [00:43<39:28,  2.20it/s]
epoch 1 iter 78: train loss 6.17545. lr 5.999177e-04:   1%|▏         | 79/5287 [00:43<39:27,  2.20it/s]
epoch 1 iter 79: train loss 6.24111. lr 5.999156e-04:   1%|▏    

...

epoch 1 iter 14702: train loss 1.10658. lr 3.467194e-04:  90%|█████████ | 14702/16329 [2:03:47<14:37,  1.85it/s]
epoch 1 iter 14702: train loss 1.10658. lr 3.467194e-04:  90%|█████████ | 14703/16329 [2:03:47<14:14,  1.90it/s]
epoch 1 iter 14703: train loss 1.11485. lr 3.466908e-04:  90%|█████████ | 14703/16329 [2:03:47<14:14,  1.90it/s]
epoch 1 iter 14703: train loss 1.11485. lr 3.466908e-04:  90%|█████████ | 14704/16329 [2:03:47<13:56,  1.94it/s]
epoch 1 iter 14704: train loss 1.08860. lr 3.466623e-04:  90%|█████████ | 14704/16329 [2:03:48<13:56,  1.94it/s]
epoch 1 iter 14704: train loss 1.08860. lr 3.466623e-04:  90%|█████████ | 14705/16329 [2:03:48<13:48,  1.96it/s]
epoch 1 iter 14705: train loss 1.09954. lr 3.466338e-04:  90%|█████████ | 14705/16329 [2:03:48<13:48,  1.96it/s]
epoch 1 iter 14705: train loss 1.09954. lr 3.466338e-04:  90%|█████████ | 14706/16329 [2:03:48<13:42,  1.97it/s]
epoch 1 iter 14706: train loss 1.07986. lr 3.466053e-04:  90%|█████████ | 14706/16329 [2:03:49<1

epoch 1 iter 14737: train loss 1.07224. lr 3.457213e-04:  90%|█████████ | 14737/16329 [2:04:04<13:17,  2.00it/s]
epoch 1 iter 14737: train loss 1.07224. lr 3.457213e-04:  90%|█████████ | 14738/16329 [2:04:04<13:13,  2.00it/s]
epoch 1 iter 14738: train loss 1.08994. lr 3.456928e-04:  90%|█████████ | 14738/16329 [2:04:05<13:13,  2.00it/s]
epoch 1 iter 14738: train loss 1.08994. lr 3.456928e-04:  90%|█████████ | 14739/16329 [2:04:05<13:12,  2.01it/s]
epoch 1 iter 14739: train loss 1.10138. lr 3.456643e-04:  90%|█████████ | 14739/16329 [2:04:05<13:12,  2.01it/s]
epoch 1 iter 14739: train loss 1.10138. lr 3.456643e-04:  90%|█████████ | 14740/16329 [2:04:05<13:10,  2.01it/s]
epoch 1 iter 14740: train loss 1.08387. lr 3.456357e-04:  90%|█████████ | 14740/16329 [2:04:06<13:10,  2.01it/s]
epoch 1 iter 14740: train loss 1.08387. lr 3.456357e-04:  90%|█████████ | 14741/16329 [2:04:06<13:08,  2.01it/s]
epoch 1 iter 14741: train loss 1.07065. lr 3.456072e-04:  90%|█████████ | 14741/16329 [2:04:06<1

epoch 1 iter 14772: train loss 1.12586. lr 3.447227e-04:  90%|█████████ | 14772/16329 [2:04:22<13:31,  1.92it/s]
epoch 1 iter 14772: train loss 1.12586. lr 3.447227e-04:  90%|█████████ | 14773/16329 [2:04:22<13:19,  1.95it/s]
epoch 1 iter 14773: train loss 1.08820. lr 3.446942e-04:  90%|█████████ | 14773/16329 [2:04:23<13:19,  1.95it/s]
epoch 1 iter 14773: train loss 1.08820. lr 3.446942e-04:  90%|█████████ | 14774/16329 [2:04:23<13:13,  1.96it/s]
epoch 1 iter 14774: train loss 1.10124. lr 3.446657e-04:  90%|█████████ | 14774/16329 [2:04:23<13:13,  1.96it/s]
epoch 1 iter 14774: train loss 1.10124. lr 3.446657e-04:  90%|█████████ | 14775/16329 [2:04:23<13:05,  1.98it/s]
epoch 1 iter 14775: train loss 1.10261. lr 3.446371e-04:  90%|█████████ | 14775/16329 [2:04:24<13:05,  1.98it/s]
epoch 1 iter 14775: train loss 1.10261. lr 3.446371e-04:  90%|█████████ | 14776/16329 [2:04:24<12:59,  1.99it/s]
epoch 1 iter 14776: train loss 1.08837. lr 3.446086e-04:  90%|█████████ | 14776/16329 [2:04:24<1

epoch 1 iter 14807: train loss 1.12266. lr 3.437237e-04:  91%|█████████ | 14807/16329 [2:04:40<13:21,  1.90it/s]
epoch 1 iter 14807: train loss 1.12266. lr 3.437237e-04:  91%|█████████ | 14808/16329 [2:04:40<13:10,  1.92it/s]
epoch 1 iter 14808: train loss 1.08393. lr 3.436951e-04:  91%|█████████ | 14808/16329 [2:04:41<13:10,  1.92it/s]
epoch 1 iter 14808: train loss 1.08393. lr 3.436951e-04:  91%|█████████ | 14809/16329 [2:04:41<13:02,  1.94it/s]
epoch 1 iter 14809: train loss 1.09962. lr 3.436666e-04:  91%|█████████ | 14809/16329 [2:04:41<13:02,  1.94it/s]
epoch 1 iter 14809: train loss 1.09962. lr 3.436666e-04:  91%|█████████ | 14810/16329 [2:04:41<12:51,  1.97it/s]
epoch 1 iter 14810: train loss 1.07891. lr 3.436380e-04:  91%|█████████ | 14810/16329 [2:04:42<12:51,  1.97it/s]
epoch 1 iter 14810: train loss 1.07891. lr 3.436380e-04:  91%|█████████ | 14811/16329 [2:04:42<12:44,  1.99it/s]
epoch 1 iter 14811: train loss 1.07368. lr 3.436095e-04:  91%|█████████ | 14811/16329 [2:04:42<1

epoch 1 iter 14842: train loss 1.10611. lr 3.427241e-04:  91%|█████████ | 14842/16329 [2:04:58<12:24,  2.00it/s]
epoch 1 iter 14842: train loss 1.10611. lr 3.427241e-04:  91%|█████████ | 14843/16329 [2:04:58<12:20,  2.01it/s]
epoch 1 iter 14843: train loss 1.12382. lr 3.426955e-04:  91%|█████████ | 14843/16329 [2:04:58<12:20,  2.01it/s]
epoch 1 iter 14843: train loss 1.12382. lr 3.426955e-04:  91%|█████████ | 14844/16329 [2:04:58<12:17,  2.01it/s]
epoch 1 iter 14844: train loss 1.10368. lr 3.426670e-04:  91%|█████████ | 14844/16329 [2:04:59<12:17,  2.01it/s]
epoch 1 iter 14844: train loss 1.10368. lr 3.426670e-04:  91%|█████████ | 14845/16329 [2:04:59<12:17,  2.01it/s]
epoch 1 iter 14845: train loss 1.09262. lr 3.426384e-04:  91%|█████████ | 14845/16329 [2:04:59<12:17,  2.01it/s]
epoch 1 iter 14845: train loss 1.09262. lr 3.426384e-04:  91%|█████████ | 14846/16329 [2:04:59<12:15,  2.02it/s]
epoch 1 iter 14846: train loss 1.11365. lr 3.426098e-04:  91%|█████████ | 14846/16329 [2:05:00<1

1
